In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from helpers import preprocessing_pipeline, count_syntactic_features

In [ ]:
raw_train_set = pd.read_csv("data/train.csv")
train_set = preprocessing_pipeline(raw_train_set)


In [ ]:
existing_column_names = columns=train_set.columns.to_list()
new_column_names = ["neg", "neu", "pos", "compound", 'Stopwords', 'Nouns', 'Verbs', 'Adverbs', 'Adjectives', 'Pronouns', "length"]
train_set = train_set.reindex(columns=existing_column_names + new_column_names)
train_set.head()


In [ ]:
for index, row in train_set.iterrows():
    tweet = row[0]
    scores = SentimentIntensityAnalyzer().polarity_scores(tweet)
    
    for sentiment, score in scores.items():
        train_set.loc[index, sentiment] = score
        
    syntax_counts = count_syntactic_features(tweet)
    for syntax, count in syntax_counts.items():
        train_set.loc[index, syntax] = count
        
    train_set.loc[index, "length"] = len(tweet)
    
train_set = train_set.drop_duplicates() 
train_set.head()

In [ ]:
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(train_set["tweets"])
bow_data = pd.DataFrame(bag_of_words.toarray(), columns=vectorizer.get_feature_names())
bow_data.head()

In [ ]:
y = train_set["class"]
train_set.drop(columns=["tweets", "class"], inplace=True)
X = pd.concat(bow_data, train_set)
X.head()